In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
%load_ext autoreload
%autoreload 2

from DLC_for_WBFM.utils.feature_detection.utils_features import *
from DLC_for_WBFM.utils.feature_detection.utils_tracklets import *
from DLC_for_WBFM.utils.feature_detection.utils_detection import *
from DLC_for_WBFM.utils.feature_detection.visualization_tracks import *
from DLC_for_WBFM.utils.feature_detection.feature_pipeline import *
from DLC_for_WBFM.utils.feature_detection.utils_affine import *

In [13]:
# Get the 3d bigtiff folder
bigtiff_folder = r'D:\More-stabilized-wbfm'

btf_fname_red = r'test2020-10-22_16-15-20_test4-channel-0-pco_camera1\test2020-10-22_16-15-20_test4-channel-0-pco_camera1bigtiff.btf'
btf_fname_red = os.path.join(bigtiff_folder, btf_fname_red)

# Annotations in the same folder as the .btf
external_annotations = r'C:\Users\charles.fieseler\Documents\Current_work\DLC_for_WBFM\DLC_for_WBFM\scratch\Charlie-test-2021_04_14\1-segmentation\metadata_500.pickle'
external_annotations = os.path.join(bigtiff_folder, external_annotations)

In [30]:
p = PreprocessingSettings()
p.do_filtering=False
p.do_rigid_alignment=True
p.do_mini_max_projection=True

# Get two sets of matches: affine, and feature-based
opt = {'start_frame':100,
       'num_frames':5,
       'num_slices':33,
      'use_affine_matching':False,
       'add_affine_to_candidates':True,
    'add_gp_to_candidates':True,
    'save_candidate_matches':True,
       'external_detections':external_annotations}
out = track_neurons_full_video(btf_fname_red, **opt, preprocessing_settings=p)


  0%|                                                                                            | 0/4 [00:00<?, ?it/s]C:\Users\charles.fieseler\Anaconda3\envs\track_using_features\lib\site-packages\sklearn\gaussian_process\kernels.py:409: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__sigma_0 is close to the specified lower bound 0.001. Decreasing the bound and calling fit again may find a better value.
  ConvergenceWarning)
C:\Users\charles.fieseler\Anaconda3\envs\track_using_features\lib\site-packages\sklearn\gaussian_process\kernels.py:409: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 1e-08. Decreasing the bound and calling fit again may find a better value.
  ConvergenceWarning)
 25%|█████████████████████                                                               | 1/4 [00:08<00:24,  8.31s/it]C:\Users\charles.fieseler\Anaconda3\envs\track_using_features\lib\site-pack

In [31]:
b_matches, b_conf, b_frames, b_candidates = out
# debug float neuron names
new_candidates = {}
for k, these_matches in b_candidates.items():
    new_matches = []
    for m in these_matches:
        if len(m)==3:
            m = (int(m[0]), int(m[1]), m[2])
        else:
            m = (int(m[0]), int(m[1]), 1.0)
        new_matches.append(m)
    new_candidates[k] = new_matches

In [32]:
bp_matches = calc_all_bipartite_matches(new_candidates)

In [33]:
df_b = build_tracklets_from_classes(b_frames, bp_matches)

# Are the matches working?

In [34]:
df_b

,clust_ind,all_ind_local,all_ind_global,all_xyz,all_prob,slice_ind,extended_this_slice,not_finished
0,0,"[50, 65, 79, 51]","[209, 383, 589, 764]","[[10.170263788968825, 296.17026378896884, 248....","[1.0, 1.0, 1.0]","[100, 101, 102, 103]",True,True
1,1,"[157, 114, 111, 93]","[316, 432, 621, 806]","[[22.064881565396497, 531.4644696189496, 279.4...","[1.0, 1.0, 1.0]","[100, 101, 102, 103]",True,True
2,2,"[137, 171, 178, 163]","[296, 489, 688, 876]","[[28.627705627705627, 441.35209235209237, 292....","[1.0, 1.0, 1.0]","[100, 101, 102, 103]",True,True
3,3,"[0, 37, 42, 4]","[159, 355, 552, 717]","[[5.345731191885038, 357.1606086221471, 288.42...","[1.0, 1.0, 1.0]","[100, 101, 102, 103]",True,True
4,4,"[148, 185, 194, 108]","[307, 503, 704, 821]","[[19.183802412406664, 332.11085582998277, 288....","[1.0, 1.0, 1.0]","[100, 101, 102, 103]",True,True
...,...,...,...,...,...,...,...,...
158,158,"[112, 79]","[622, 792]","[[19.501168224299064, 663.3317757009346, 413.9...",[1.0],"[102, 103]",True,True
159,159,"[124, 91]","[634, 804]","[[18.70026525198939, 513.2440318302388, 73.779...",[1.0],"[102, 103]",True,True
160,160,"[94, 165]","[604, 878]","[[15.40681362725451, 474.47895791583164, 100.9...",[1.0],"[102, 103]",True,True
161,161,"[113, 95]","[623, 808]","[[20.277268093781856, 676.743119266055, 415.05...",[1.0],"[102, 103]",True,True


# Counting errors with external annotations ?

In [38]:
with open(external_annotations, 'rb') as f:
    ann = pickle.load(f)

In [40]:
ann[100]

,total_brightness,neuron_volume,centroids
60,450856,1183,"(5.345731191885038, 288.42349957734575, 357.16..."
93,105268,467,"(2.8394004282655247, 296.48608137044965, 710.3..."
102,83973,478,"(4.489539748953975, 289.9539748953975, 418.133..."
103,226888,929,"(5.898815931108719, 227.75457481162542, 311.12..."
105,371235,1141,"(8.946538124452236, 182.14110429447854, 309.13..."
...,...,...,...
457,463298,1364,"(20.169354838709676, 255.31304985337243, 374.7..."
458,410660,1272,"(21.10377358490566, 251.75471698113208, 364.89..."
459,320934,1126,"(20.91740674955595, 272.8916518650089, 476.667..."
460,236421,971,"(22.064881565396497, 279.48609680741504, 531.4..."
